# 1. 동적 크롤링을 통해 음원 순위 데이터 수집하기

### 00. 크롬 드라이버 열기 

In [4]:
# 전체화면 아닐 때 예외처리 완료
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time as time
from bs4 import BeautifulSoup as bs
import re

In [5]:
# 크롬드라이버 열기
driver = webdriver.Chrome()
driver.maximize_window() # 크롬창 크기 최대

# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)


### 01. 차트 파인더 조건 클릭하기

In [6]:
#차트파인더 클릭
driver.find_element(By.CSS_SELECTOR, 'div.chart_finder').click()
time.sleep(2)

#주간차트 클릭
driver.find_element(By.CSS_SELECTOR, 'h4.tab01').click()
time.sleep(2)

# 연대선택 2020년 클릭
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label').click()
time.sleep(2)

# 연대선택 2023년 클릭
# 규칙성이 있기 때문에 2024년은 li[1] 2023년은  li[2] 2022년은 li[3]으로 바꿔주면 됨.
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label').click()
time.sleep(2)

#  1월 클릭
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label').click()
time.sleep(2)

#  첫째주 클릭
driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[1]/span/label').click()



 ### 02. 클릭 버튼 활성화된 month까지 가져오도록  month_active_list 만들어주기

In [7]:
# 2023년은 12월까지 결과가 다 나왔기 때문에 month_active_list가 01월부터 12월까지 만들어진다. 
# 2023포함 이전 연도는 12월까지 결과가 다 나왔기 때문에  12가 출력되고, # 2024년은 2월까지만 출력된다

month_active_list = driver.find_element(By.CSS_SELECTOR, 'div.box_chic.nth3.view.on > div.list_value').text
active_last_month = int(month_active_list.split('\n')[-1].replace('월','').strip()) # 2023년 활성화된 마지막월은 12
active_last_month

12

### 03. 클릭 버튼 활성화된 week까지 가져오도록 week_active_list 만들어주기

In [8]:
active_week_list = driver.find_element(By.CSS_SELECTOR, 'div.box_chic.nth4.view.on > div.list_value > ul').text
active_last_week = active_week_list.count('\n')+1 #\n의 개수를 세서  +1하여 활성화된 주차를 계산함.
active_last_week # 1월은 4주까지 있으므로 4주까지 클릭됨

4

### 03. 12월까지 반복수행 하여 빈 리스트에 지정한 값을 담는다

In [9]:


week_list = []
rank_list = []
title_list = []
artist_list = []
album_list = []

# 12월까지  12번 반복해서 월, 장르, 검색 버튼 클릭하고 연도와 월 가져오는 for문
for month in range(active_last_month) :  
 
    #월 버튼 클릭 
    month_tab = f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{month+1}]/span/label' 
    driver.find_element(By.XPATH, month_tab).click()
    time.sleep(2)
    
    for week in range(active_last_week) : 
    #주 버튼 클릭 
        week_tab = f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[{week+1}]/span/label'
        driver.find_element(By.XPATH, week_tab).click()
        time.sleep(2)
    
        
        # 장르 종합 선택
        driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
        time.sleep(2)
    
        # 검색 버튼 클릭
        driver.find_element(By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span/span').click()
        time.sleep(2)
       
        #해당 주의 정보 파싱해오기 
        html = driver.page_source
        soup = bs(html, 'html.parser')
        song_soup = soup.select('tbody > tr')
        week_soup = soup.select('span.datelk')[0].get_text().replace('\n', '')
    
        # 해당 월마다 rank, title, artist, album 정보 가져오는 for문 
        for song in song_soup :
        # list에 append하면서 쓸데없는 문자, 공백 replace를 통해 삭제
            rank = song.find('span', class_= 'rank').get_text().replace('\n', '')
            title = song.find('div', class_= 'ellipsis rank01').get_text().replace('\n', '')
            artist = song.find('a', class_= 'fc_mgray').get_text()
            album = song.find('div', class_= 'ellipsis rank03').get_text().replace('\n', '')
            rank_list.append(rank)
            title_list.append(title)
            artist_list.append(artist)
            album_list.append(album)
            week_list.append(week_soup)
        
     

chart_df_23 = pd.DataFrame(columns=['week','rank','title','artist','album'],
                         data=zip(week_list,rank_list, title_list, artist_list, album_list))
        
    
    


# 2. 전처리 

### 초동(음반)데이터와 크롤링으로 수집한 멜론 주간차트 데이터 merge

In [338]:
df_merge = pd.merge(df_chodong , df, how='left', on= 'artist' )
df_merge.head()
df_merge.drop(['Unnamed: 0','title_y','album_y'], axis=1, inplace=True)
df_merge.head()


,artist,sex,week_x,title_x,album_x,chodong,melon_like,week_y,rank
0,세븐틴 (SEVENTEEN),m,2023.04.24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.04.24 ~ 2023.04.30,8
1,세븐틴 (SEVENTEEN),m,2023.04.24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.01 ~ 2023.05.07,5
2,세븐틴 (SEVENTEEN),m,2023.04.24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.08 ~ 2023.05.14,5
3,세븐틴 (SEVENTEEN),m,2023.04.24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.15 ~ 2023.05.21,6
4,세븐틴 (SEVENTEEN),m,2023.04.24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.22 ~ 2023.05.28,7


### 발매일로부터 차트인 진입에 소요된 기간 알아보기 위해 week_y컬럼의 공백 기준 구분

In [339]:
df_merge['week_end'] = df_merge["week_y"].str.split(" ",expand=True)[2]
df_merge['week_start'] = df_merge["week_y"].str.split(" ",expand=True)[0]

In [340]:
df_merge['week_end']= pd.to_datetime(df_merge['week_end'])
df_merge['week_x']= pd.to_datetime(df_merge['week_x'])
df_merge['week_start']= pd.to_datetime(df_merge['week_start'])

In [341]:
df_merge['duration'] =  df_merge["week_start"] - df_merge["week_x"]
df_merge

,artist,sex,week_x,title_x,album_x,chodong,melon_like,week_y,rank,week_end,week_start,duration
0,세븐틴 (SEVENTEEN),m,2023-04-24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.04.24 ~ 2023.04.30,8,2023-04-30,2023-04-24,0 days
1,세븐틴 (SEVENTEEN),m,2023-04-24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.01 ~ 2023.05.07,5,2023-05-07,2023-05-01,7 days
2,세븐틴 (SEVENTEEN),m,2023-04-24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.08 ~ 2023.05.14,5,2023-05-14,2023-05-08,14 days
3,세븐틴 (SEVENTEEN),m,2023-04-24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.15 ~ 2023.05.21,6,2023-05-21,2023-05-15,21 days
4,세븐틴 (SEVENTEEN),m,2023-04-24,손오공,SEVENTEEN 10th MiniAlbum ‘FML',4550200,169085,2023.05.22 ~ 2023.05.28,7,2023-05-28,2023-05-22,28 days
...,...,...,...,...,...,...,...,...,...,...,...,...
336,ITZY (있지),f,2023-07-31,CAKE,KILL MY DOUBT,823700,34307,2023.10.23 ~ 2023.10.29,44,2023-10-29,2023-10-23,84 days
337,ITZY (있지),f,2023-07-31,CAKE,KILL MY DOUBT,823700,34307,2023.10.30 ~ 2023.11.05,52,2023-11-05,2023-10-30,91 days
338,ITZY (있지),f,2023-07-31,CAKE,KILL MY DOUBT,823700,34307,2023.11.06 ~ 2023.11.12,61,2023-11-12,2023-11-06,98 days
339,ITZY (있지),f,2023-07-31,CAKE,KILL MY DOUBT,823700,34307,2023.11.13 ~ 2023.11.19,74,2023-11-19,2023-11-13,105 days


In [342]:
import numpy as np
df_merge['rank'].replace(np.NaN, 101, inplace = True) # 100위에 차트인하지 못한 경우는 101위로 만들어줌

In [343]:
df_duration = df_merge.groupby("artist")['duration'].min().reset_index().sort_values(by = 'duration')

In [ ]:
df_duration.loc[(df_duration['duration'] <= '0 days'), 'week'] = '1'
df_duration.loc[(df_duration['duration'] <= '0 days') | (df_duration['duration'] < '7 days') , 'week'] = '1'
df_duration.loc[(df_duration['duration'] == '7 days'), 'week'] = '2'
df_duration.loc[(df_duration['duration'] >= '45 days'), 'week'] = '7'
df_duration

In [346]:
df_duration.drop(['duration'], axis=1, inplace=True)
df_duration

,artist,week
1,Agust D,1
8,NewJeans,1
14,정국,1
11,STAYC(스테이씨),1
13,세븐틴 (SEVENTEEN),1
12,aespa,1
10,Red Velvet (레드벨벳),1
0,(여자)아이들,1
6,LE SSERAFIM (르세라핌),1
7,NMIXX,1


In [354]:
df_merge = df_merge.sort_values(by = 'week_y')

# 3. 시각화

In [348]:

import pandas as pd
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [356]:
fig = px.line(data_frame = df_merge
            ,x='week_y', y='rank', color = 'artist'
            ,markers=True
            ,color_discrete_sequence= px.colors.qualitative.Pastel
            ,width=1000, height=800
            ,title="[컴백 후 주차별 랭크 추이]")
           
            

fig.update_layout(
                 title_x = 0.5,
                 title_y = 0.9,
                 title_xanchor = "center",
                 title_yanchor = "middle",
                 # 폰트 스타일 추가 부분
                 title_font_size = 16,
                 template='plotly_dark')

fig.update_xaxes(title_text='')
fig.update_yaxes(autorange="reversed") #y축을 역으로 해서 1위가 상단, 100위가 하단으로 향하게

fig.add_annotation( 
            x= '2023.10.23 ~ 2023.10.29' , y= '30',
            text= "다음 앨범 발매로 인한 순위 반등",
            showarrow=True,
            arrowhead=1,
            font_color = 'red',
            arrowcolor= 'red')


fig.show()



In [47]:
df_2023 = df_2023.sort_values(by= 'week')
df_2023.to_csv('/Users/yeon/Desktop/Portfolio/하이브/weekly_df.csv')